## 1. 建立並保存 ONNX 模型檔案
以下是一個使用 TensorFlow 建立鳶尾花（Iris）分類模型並將其導出為 ONNX 格式的範例。該模型使用簡單的全連接層來進行分類，並轉換為 ONNX 格式，方便在 TVM 或其他 ONNX 支持的推理引擎上運行。

### 1.1 安裝必要的套件
如果尚未安裝 tensorflow 和 tf2onnx，可以使用以下命令安裝：



In [ ]:
!pip install tensorflow tf2onnx

### 1.2 建立並訓練 TensorFlow 模型
以下程式碼將建立一個簡單的神經網絡來分類鳶尾花數據集，並將其導出為 ONNX 格式。

In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

# 載入鳶尾花資料集
iris = load_iris()
X = iris.data.astype(np.float32)
y = iris.target

# 分割資料集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 建立模型
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(4,)),  # 4 個特徵
    tf.keras.layers.Dense(10, activation='relu'),  # 隱藏層
    tf.keras.layers.Dense(3, activation='softmax') # 輸出層，3 個分類
])

# 編譯模型
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 訓練模型
model.fit(X_train, y_train, epochs=50, batch_size=5, verbose=0)

# 評估模型
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"模型準確率: {accuracy:.2f}")

### 1.3 將模型轉換為 ONNX 格式
使用 tf2onnx 將訓練好的 TensorFlow 模型轉換為 ONNX 格式：

In [ ]:
import tf2onnx

# 將 Keras 模型轉換為 ONNX 格式
spec = (tf.TensorSpec((None, 4), tf.float32, name="float_input"),)  # 定義輸入規範
output_path = "tf_model.onnx"  # 輸出 ONNX 模型的路徑

# 轉換模型
model_proto, _ = tf2onnx.convert.from_keras(model, input_signature=spec, opset=13)
with open(output_path, "wb") as f:
    f.write(model_proto.SerializeToString())

print(f"ONNX 模型已保存至 {output_path}")

In [1]:
import onnxruntime as ort
import numpy as np

# 加載 ONNX 模型
session = ort.InferenceSession('tf_model.onnx')

# 準備輸入資料
input_name = session.get_inputs()[0].name
input_data = np.array([[6.3, 3.3, 6.0, 2.5]], dtype=np.float32)

# 進行推理
pred_onnx = session.run(None, {input_name: input_data})

# 輸出預測結果
print(pred_onnx)

[array([[5.2434858e-04, 8.7534554e-02, 9.1194111e-01]], dtype=float32)]


## 2. 使用 TVM 轉換模型為共享庫
### 2.1 將 ONNX 模型編譯為共享庫
使用 TVM relay 將 tf_model.onnx 模型轉換為共享庫（.so 文件）。

> 成功輸出後即可前往第3撰寫 C++ 程式進行推論

#### 補充

在 TVM 中，`opt_level` 參數用於控制編譯過程中的優化程度。不同的優化 Pass（即優化步驟）被分配到不同的優化等級。當您設定特定的 `opt_level` 時，TVM 會執行該等級及以下所有對應的優化 Pass。

**`opt_level=0`**
執行基本的優化：
- `SimplifyInference`：簡化推理過程。

**`opt_level=1`**
包含等級 0 的優化，並新增：
- `OpFusion`：運算符融合，將多個運算符合併以提高效率。

**`opt_level=2`**
包含等級 0 和 1 的優化，並新增：
- `FoldConstant`：常量折疊，預先計算常量表達式以減少運算量。

**`opt_level=3`**
包含等級 0、1 和 2 的優化，並新增：
- `FoldScaleAxis`：折疊縮放軸。
- `AlterOpLayout`：改變運算符的佈局以適應特定硬體。
- `CanonicalizeOps`：規範化運算符。
- `CanonicalizeCast`：規範化類型轉換。
- `EliminateCommonSubexpr`：消除公共子表達式以減少重複計算。

**`opt_level=4`**
包含等級 0 至 3 的優化，並新增：
- `CombineParallelConv2D`：合併平行的 2D 卷積運算。
- `CombineParallelDense`：合併平行的全連接層運算。
- `CombineParallelBatchMatmul`：合併平行的批次矩陣乘法運算。
- `FastMath`：啟用快速數學運算，可能會犧牲精度以換取速度。

- 參考:[tvm.relay.build_config](https://tvm.apache.org/docs/reference/api/python/relay/index.html)


In [5]:
%%time
import tvm
from tvm import relay
from tvm.contrib import cc, utils
from tvm.contrib import graph_executor
import onnx

# 載入 ONNX 模型
onnx_model = onnx.load("tf_model.onnx")

# 將 ONNX 模型轉換為 Relay 模型
input_name = 'float_input'  # 輸入名稱可在 ONNX 模型中確認
shape_dict = {input_name: (1, 4)}
mod, params = relay.frontend.from_onnx(onnx_model, shape_dict)

# 設置目標架構，這裡假設為通用的 CPU
target = tvm.target.Target("llvm", host="llvm -mtriple=x86_64-linux-gnu")
# target = tvm.target.Target("llvm", host="llvm -mtriple=aarch64-linux-gnu")

with tvm.transform.PassContext(opt_level=0):
    lib = relay.build(mod, target, params=params)

# 編譯輸出共享庫 
lib.export_library("output.so", cc="gcc")
# lib.export_library("output.so", cc="aarch64-linux-gnu-gcc")

CPU times: user 2.2 s, sys: 2.08 s, total: 4.28 s
Wall time: 489 ms


In [4]:
!du -h output.so

92K	output.so


## 2.2 使用 TVM Python API 推論(optional)
使用 TVM runtime 載入共享庫並設置輸入數據，即可執行推論。

In [2]:
import tvm
from tvm.contrib import graph_executor
import numpy as np

# 在目標設備上載入共享庫
loaded_lib = tvm.runtime.load_module("output.so")
module = graph_executor.GraphModule(loaded_lib["default"](tvm.cpu()))

# 準備輸入資料
input_data = np.array([[6.3, 3.3, 6.0, 2.5]], dtype=np.float32)
# # 設定輸入數據並執行推論
module.set_input("float_input", tvm.nd.array(input_data))
module.run()
output = module.get_output(0).asnumpy()
output

array([[5.243493e-04, 8.753461e-02, 9.119410e-01]], dtype=float32)

In [3]:
loaded_lib

Module(GraphExecutorFactory, 558a6671faf8)

## 3. 撰寫 C++ 程式進行推論

### 3.1 撰寫 C++ 程式

> 請參考 tf-example/tf_inference.cpp

### 3.2 編譯程式

In [ ]:
!g++ -std=c++17 -o main tf_inference.cpp \
    -I../../tvm/include \
    -I../../tvm/3rdparty/dlpack/include \
    -I../../tvm/3rdparty/dmlc-core/include \
    ../../tvm/build/libtvm_runtime.so \
    -ldl -pthread

In [ ]:
!g++ -std=c++17 -o main tf_inference.cpp \
    -I../../tvm/include \
    -I../../tvm/3rdparty/dlpack/include \
    -I../../tvm/3rdparty/dmlc-core/include \
    -ltvm_runtime -ldl -pthread

In [1]:
!export LD_LIBRARY_PATH=$(pwd):$LD_LIBRARY_PAT

In [1]:
!./main

Prediction Probabilities: [0.000524349, 0.0875346, 0.911941]


In [6]:
# 檢查相依庫
!ldd ./output.so

	statically linked


In [3]:
# 檢查共享庫位置
!ldconfig -p | grep libtvm_runtime.so

	libtvm_runtime.so (libc6,x86-64) => /usr/local/lib/libtvm_runtime.so


## Reference
- [TVM 編譯 ONNX 模型](https://tvm.hyper.ai/docs/how_to/compile/compile_onnx)

## TVM優化
在使用 TVM 編譯模型時，出現了以下警告訊息：

```
One or more operators have not been tuned. Please tune your model for better performance. Use DEBUG logging level to see more details.

```

這個警告的含義是：
- 模型中有一個或多個算子（operators）尚未經過調優（tuning），因此 TVM 使用了預設的調度（schedule）來生成程式碼。
- 這些預設的調度可能不是針對您的硬體或模型特性進行優化的，因此可能無法達到最佳性能。
- 建議對模型進行調優，以獲得更好的性能。

為了解決這個警告並提高模型的性能，您需要使用 TVM 的自動調優工具（AutoTVM 或 AutoScheduler）對模型進行算子級別的調優。

### 方法一：使用 AutoTVM 進行調優

AutoTVM 是 TVM 提供的自動調優框架，可以自動搜尋最佳的算子實現，以提高模型的運行性能。

**步驟**：
1. 定義調優任務
2. 配置調優選項
3. 運行調優任務
4. 使用調優結果編譯模型

#### 1. 定義調優任務

In [4]:
import tvm
from tvm import relay, autotvm
from tvm.contrib import graph_executor
import onnx

# 載入 ONNX 模型
onnx_model = onnx.load("tf_model.onnx")

# 將 ONNX 模型轉換為 Relay 模型
input_name = 'float_input'  # 輸入名稱可在 ONNX 模型中確認
shape_dict = {input_name: (1, 4)}
mod, params = relay.frontend.from_onnx(onnx_model, shape_dict)

# 設置目標架構，這裡假設為通用的 CPU
target = "llvm"

# 定義調優任務
tasks = autotvm.task.extract_from_program(mod["main"], target=target, params=params)

#### 2. 配置調優選項

In [5]:
# 定義日誌文件，用於保存調優記錄
log_file = "tuning.log"

# 定義測量選項
measure_option = autotvm.measure_option(
    builder=autotvm.LocalBuilder(),
    runner=autotvm.LocalRunner(number=10, repeat=1, timeout=10, min_repeat_ms=100)
)

#### 3. 運行調優任務
注意 n_trial 是調優的總次數，調優次數越多，找到更優解的可能性越大，但耗時也會增加。您可以根據需要調整。

In [ ]:
from tvm.autotvm.tuner import XGBTuner

# 對每個任務進行調優
for i, task in enumerate(tasks):
    print(f"===== Tuning task {i+1}/{len(tasks)}: {task.name} =====")
    tuner = XGBTuner(task)
    tuner.tune(
        n_trial=100,  # 調優次數，可以根據需要調整
        measure_option=measure_option,
        callbacks=[
            autotvm.callback.progress_bar(100),
            autotvm.callback.log_to_file(log_file)
        ]
    )


#### 4. 使用調優結果編譯模型

In [14]:
# 套用調優結果
with autotvm.apply_history_best(log_file):
    with tvm.transform.PassContext(opt_level=3):
        lib = relay.build(mod, target=target, params=params)

# 編譯輸出共享庫
lib.export_library("output.so") #優化結果 64K

In [15]:
!du -h output.so

64K	output.so


### 方法二：使用 AutoScheduler 進行調優
AutoScheduler 是 TVM 新一代的自動調優框架，能夠更高效地探索優化空間，適用於複雜模型。

**步驟**：
1. 定義調優任務
2. 運行調優
3. 使用調優結果編譯模型

#### 1. 定義調優任務

In [ ]:
from tvm import auto_scheduler

# 設置目標架構
target = tvm.target.Target("llvm")

# 提取調優任務
tasks, task_weights = auto_scheduler.extract_tasks(mod["main"], params, target)


#### 2. 運行調優

In [ ]:
log_file = "auto_scheduler_tuning.json"

def tune_tasks(tasks, task_weights, log_file):
    tuner = auto_scheduler.TaskScheduler(tasks, task_weights)
    tune_option = auto_scheduler.TuningOptions(
        num_measure_trials=100,  # 調優總次數，可以調整
        runner=auto_scheduler.LocalRunner(repeat=1, min_repeat_ms=300, timeout=10),
        measure_callbacks=[auto_scheduler.RecordToFile(log_file)],
    )
    tuner.tune(tune_option)

# 開始調優
tune_tasks(tasks, task_weights, log_file)

#### 3. 使用調優結果編譯模型

In [ ]:
# 使用調優日誌
with auto_scheduler.ApplyHistoryBest(log_file):
    with tvm.transform.PassContext(opt_level=3,
                                   config={"relay.backend.use_auto_scheduler": True}):
        lib = relay.build(mod, target=target, params=params)

# 編譯輸出共享庫
lib.export_library("output.so")


### 1.4 TVM 進行編譯產生 C(Bug)

In [ ]:
!export TVM_HOME=/home/jovyan/project/ONNX-MLIR/tvm
!export PYTHONPATH=$TVM_HOME/python:$PYTHONPATH

In [ ]:
!TVM_LIBRARY_PATH=/home/jovyan/project/ONNX-MLIR/tvm/build python3 run.py

In [1]:
import onnx
import tvm
from tvm import relay

# 載入 ONNX 模型
onnx_model = onnx.load("./tf_model.onnx")

# 定義輸入資訊
input_name = 'float_input'
shape_dict = {input_name: (1, 4)}

# 將 ONNX 模型轉換為 Relay 模組
mod, params = relay.frontend.from_onnx(onnx_model, shape_dict)

# 設定編譯目標為產生 C 原始碼
target = tvm.target.Target("c")

# 使用 AOT 執行器
executor = tvm.relay.build_module.Executor("aot")

# 編譯 Relay 模組
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, target=target, executor=executor, params=params)


[14:49:59] /home/jovyan/project/ONNX-MLIR/tvm/src/target/llvm/llvm_instance.cc:226: Error: Using LLVM 19.1.3 with `-mcpu=apple-latest` is not valid in `-mtriple=arm64-apple-macos`, using default `-mcpu=generic`
[14:49:59] /home/jovyan/project/ONNX-MLIR/tvm/src/target/llvm/llvm_instance.cc:226: Error: Using LLVM 19.1.3 with `-mcpu=apple-latest` is not valid in `-mtriple=arm64-apple-macos`, using default `-mcpu=generic`
[14:49:59] /home/jovyan/project/ONNX-MLIR/tvm/src/target/llvm/llvm_instance.cc:226: Error: Using LLVM 19.1.3 with `-mcpu=apple-latest` is not valid in `-mtriple=arm64-apple-macos`, using default `-mcpu=generic`
One or more operators have not been tuned. Please tune your model for better performance. Use DEBUG logging level to see more details.


## 測試有Bug

In [ ]:
import tvm
from tvm import relay
from tvm.contrib import cc
import onnx

# 載入 ONNX 模型
onnx_model = onnx.load("tf_model.onnx")

# 將 ONNX 模型轉換為 Relay 模型
input_name = 'float_input'
shape_dict = {input_name: (1, 4)}
mod, params = relay.frontend.from_onnx(onnx_model, shape_dict)

# 設置目標架構
target = tvm.target.Target("c")

with tvm.transform.PassContext(opt_level=1):
    # 編譯 Relay 模型
    mod = relay.transform.SimplifyInference()(mod)
    lib = relay.build(mod, target=target, params=params)

# Export the compiled library
c_source_code = lib.get_lib().get_source()

# 將程式碼寫入到一個 .c 檔案
with open('output.c', 'w') as file:
    file.write(c_source_code)
print("C source code 已經儲存到 output.c")

# lib.export_library("model")

In [19]:
!gcc -o inference output.c -lm

In file included from output.c:3:
/usr/local/include/tvm/runtime/c_runtime_api.h:79:10: fatal error: dlpack/dlpack.h: No such file or directory
   79 | #include <dlpack/dlpack.h>
      |          ^~~~~~~~~~~~~~~~~
compilation terminated.


In [20]:
!g++ output.c -o output -I../../tvm/3rdparty/dlpack/include -ltvm_runtime -ldl -lpthread

output.c: In function ‘int32_t tvmgen_default_fused_nn_contrib_dense_pack_add(void*, int32_t*, int32_t, void*, int32_t*, void*)’:
output.c:56:3: error: ‘float3’ was not declared in this scope; did you mean ‘float’?
   56 |   float3 compute_global[1];
      |   ^~~~~~
      |   float
output.c:57:3: error: ‘compute_global’ was not declared in this scope
   57 |   compute_global[0] = ((float3)(0.000000e+00f, 0.000000e+00f, 0.000000e+00f));
      |   ^~~~~~~~~~~~~~
output.c:59:144: error: expected primary-expression before ‘)’ token
   59 | float*)p0_1)[k_outer], ((float*)p0_1)[k_outer])) * *(float3*)(((float*)p1_1) + (k_outer * 3))));
      |                                                             ^

output.c:61:12: error: expected primary-expression before ‘)’ token
   61 |   *(float3*)(((float*)T_add_1) + 0) = (compute_global[0] + *(float3*)(((float*)p2_1) + 0));
      |            ^
output.c:61:69: error: expected primary-expression before ‘)’ token
   61 | t3*)(((float*)T_add_1) +

## microTVM

In [31]:
import onnx
import tvm
from tvm import relay
from tvm.micro import export_model_library_format

# 載入 ONNX 模型
onnx_model = onnx.load("./tf_model.onnx")

# 定義輸入資訊
input_name = 'float_input'
shape_dict = {input_name: (1, 4)}

# 將 ONNX 模型轉換為 Relay 模組
mod, params = relay.frontend.from_onnx(onnx_model, shape_dict)



# 設定目標為嵌入式設備，使用 MicroTVM
target = tvm.target.target.micro("host")

with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, target=target, executor=executor, params=params)


# 導出為模型庫格式，包含所有生成的源代碼
export_model_library_format(lib, "microtvm_model.tar")

PosixPath('microtvm_model.tar')

In [38]:
!mkdir build
!tar xvf microtvm_model.tar -C build

./
./codegen/
./codegen/host/
./codegen/host/src/
./codegen/host/src/default_lib0.c
./codegen/host/src/default_lib1.c
./metadata.json
./parameters/
./parameters/default.params
./src/
./src/default.relay
